In [39]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import os
import re
from langchain_postgres.vectorstores import PGVector
from dotenv import load_dotenv


In [56]:
load_dotenv()

PGV_USER=os.getenv("PGV_USER")
PGV_PASSWORD=os.getenv("PGV_PASSWORD")
PGV_HOST=os.getenv("PGV_HOST")
PGV_PORT=os.getenv("PGV_PORT")
PGV_DATABASE_NAME=os.getenv("PGV_DATABASE_NAME")

In [30]:
CHUNK_SIZE = 2000
CHUNK_OVERLAP = 500

In [31]:
moder_for_embedding = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [32]:

direc = 'C:/Users/claud\Desktop/Cursos/LearningLangChain/Deploy/PDF'  # tener ojo con esta ruta, porque o sino no se hace el embeddign correcto
loader = PyPDFDirectoryLoader(direc)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\claud\AppData\Local\Temp\ipykernel_15340\2345177032.py:1: SyntaxWarning: invalid escape sequence '\D'
  direc = 'C:/Users/claud\Desktop/Cursos/LearningLangChain/Deploy/PDF'  # tener ojo con esta ruta, porque o sino no se hace el embeddign correcto


In [64]:
pdfs = loader.load()

for pdf in pdfs:
    pdf.page_content = re.sub(r'\n\*?\s*\n\*?', ' ', pdf.page_content)
    pdf.page_content = re.sub(r'\s+\b', ' ', pdf.page_content)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunks = text_splitter.split_documents(pdfs)
print(len(chunks))


56


In [ ]:
connection = f'postgresql+psycopg://{PGV_USER}:{PGV_PASSWORD}@{PGV_HOST}:{PGV_PORT}/{PGV_DATABASE_NAME}'

In [48]:
db = PGVector.from_documents(chunks,moder_for_embedding,connection=connection)

VectorStoreRetriever(tags=['PGVector', 'FastEmbedEmbeddings'], vectorstore=<langchain_postgres.vectorstores.PGVector object at 0x00000188B13ACB30>, search_kwargs={})

In [63]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

# Cadena de conexión para PostgreSQL (reemplaza con tus valores)
connection_str = f'postgresql+psycopg://{PGV_USER}:{PGV_PASSWORD}@{PGV_HOST}:{PGV_PORT}/{PGV_DATABASE_NAME}'

# Crear el motor de conexión con SQLAlchemy
engine = create_engine(connection_str)

# Crear una sesión para interactuar con la base de datos
Session = sessionmaker(bind=engine)
session = Session()

# Nombre de la tabla donde se almacenan los documentos en PGVector
table_name = "langchain_pg_embedding"  # Reemplaza con el nombre real de la tabla

# Ejecutar una consulta SQL para contar los documentos en la tabla
count_query = text(f"SELECT COUNT(*) FROM {table_name}")

# Ejecutar la consulta y obtener el resultado
result = session.execute(count_query).fetchone()
document_count = result[0] if result else 0

# Imprimir la cantidad de documentos
print(f"Cantidad de documentos almacenados: {document_count}")


Cantidad de documentos almacenados: 280


In [ ]:
retriever = db.as_retriever()

In [ ]:
docs= retriever.invoke("¿Que es un alumno regular?")
docs

[Document(id='e361c27b-6286-490b-9628-384aaaa302ed', metadata={'page': 4, 'source': 'C:\\Users\\claud\\Desktop\\Cursos\\LearningLangChain\\Deploy\\PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf'}, page_content='Art. 6 Estudiante regular: Es quien ingresa a través de los procedimientos de admisión, que establece el Título II| del presente Reglamento, y adscrito a un programa o carrera determinada, cursa estudios conducentes a un grado académico y/o título profesional. La calidad de estudiante regular de la Universidad Católica del Norte se adquiere una vez que la matrícula ha sido formalizada en conformidad a las normas correspondientes. Art. 7. Estudiante egresado/a: Corresponde a estudiante regular que ha aprobado las actividades curriculares establecidas en el libro de carrera o programa respectivo y a quienes solo reste cumplir con las actividades finales exigidas para obtener el grado o titulo respectivo. Art. 8. Estudiante de intercambio saliente: Estudiante de la UCN,

In [ ]:
vector_store = PGVector(
            embeddings=moder_for_embedding,
            connection=connection,
            use_jsonb=True,
        )

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
docs= retriever.invoke("¿Que es un alumno regular?")
docs

[Document(id='e361c27b-6286-490b-9628-384aaaa302ed', metadata={'page': 4, 'source': 'C:\\Users\\claud\\Desktop\\Cursos\\LearningLangChain\\Deploy\\PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf'}, page_content='Art. 6 Estudiante regular: Es quien ingresa a través de los procedimientos de admisión, que establece el Título II| del presente Reglamento, y adscrito a un programa o carrera determinada, cursa estudios conducentes a un grado académico y/o título profesional. La calidad de estudiante regular de la Universidad Católica del Norte se adquiere una vez que la matrícula ha sido formalizada en conformidad a las normas correspondientes. Art. 7. Estudiante egresado/a: Corresponde a estudiante regular que ha aprobado las actividades curriculares establecidas en el libro de carrera o programa respectivo y a quienes solo reste cumplir con las actividades finales exigidas para obtener el grado o titulo respectivo. Art. 8. Estudiante de intercambio saliente: Estudiante de la UCN,